In [1]:
flex_title = "Indonesia Fire Danger Rating System"
flex_subtitle = "Nusantara Earth Observation Network"

In [2]:
import ee
from datetime import date, timedelta

# Mapping backend
import folium
import geemap.foliumap as geemap

# Interactive Python Widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

min_dict = {
    'FWI' : 1.0, 'ISI' : 1.0, 'BUI' : 6.0,
    'DC' : 140.0, 'DMC' : 4.0, 'FFMC' : 72.0,
}

max_dict = {
    'FWI' : 13.0, 'ISI' : 5.0, 'BUI' : 33.0,
    'DC' : 350.0, 'DMC' : 29.0, 'FFMC' : 82.0,
}

palette = ['blue', 'green', 'yellow', 'red']

## Section

### Parameters

In [3]:
# Province selection
prov_label = widgets.Label(value='Province:')
prov_var = widgets.Dropdown(
    options=[
        'Sumatera Selatan',
    ],
    value='Sumatera Selatan',
)

# Observation date selection
obs_label = widgets.Label(value='Date:')
obs_var = widgets.DatePicker(
    value=date.today() - timedelta(days=1)
)

# Component selection
comp_label = widgets.Label(value='Component:')
comp_var = widgets.Dropdown(
    options=[
        'FFMC', 'DMC', 'DC', 'ISI', 'BUI', 'FWI'
    ],
    value='FWI',
)

# Create and display the sidebar
inputs = widgets.VBox([prov_label, prov_var, obs_label, obs_var, comp_label, comp_var])
display(inputs)

### About

This application shows calibrated Canadian Fire Weather Index (FWI) calculated by using data from GSMaP (JAXA) and GFS (NOAA).

For more information, contact:
[Josef Matondang](mailto:admin@josefmtd.com)

## Column

### Fire Weather Index

In [5]:
def on_value_change(change):
    obs = obs_var.value
    comp = comp_var.value
    prov = prov_var.value

    with out:
        clear_output(wait=True)

        # Create folium map object
        fig = folium.Figure(height="100%")
        Map = geemap.Map(center=(-3.1, 104.1), zoom=7,
            add_google_map=False).add_to(fig)

        # Obtain ee.Image from Google Cloud Storage
        url = f'gs://fwi-bicubic-outputs/ID-SS_{obs.isoformat()}_FWI.tif'
        fwi_data = ee.Image.loadGeoTIFF(url)

        # Visualization parameters
        min_val = min_dict[comp]
        max_val = max_dict[comp]
        vis_params = {
            'min': min_val, 
            'max': max_val, 
            'palette': palette
        }

        # Add province parameters
        province = ee.FeatureCollection('FAO/GAUL/2015/level2') \
            .filter(ee.Filter.eq('ADM1_NAME', prov))

        # Add overlay layer
        Map.addLayer(fwi_data.select(comp), vis_params, comp)
        Map.addLayer(province, {}, prov)
        Map.add_colorbar(colors=palette, vmin=min_val, vmax=max_val)
        display(Map)

# Create output figure and callback functions to update the values
out = widgets.Output()
prov_var.observe(on_value_change, names="value")
obs_var.observe(on_value_change, names="value")
comp_var.observe(on_value_change, names="value")

In [6]:
# Shows the figure
on_value_change(None)
display(out)

Output()